# 实验一
自定义VGG16网络，并在Kaggle猫/狗数据集上进行训练和测试

## 1.加载keras模块

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import*
from tensorflow.keras.models import*
#from keras.models import Sequential
#from keras.layers import Conv2D, MaxPooling2D
#from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
import numpy as np

Using TensorFlow backend.


### 定义CNN网络结构



In [2]:
img_width, img_height = 224,224
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
    
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=input_shape))
model.add(Conv2D(64, (3,3), activation='relu'))
#model.add(Conv2D(64, (3,3), activation='relu',input_shape=input_shape))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64,(3,3), activation='relu'))
#model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(MaxPool2D(pool_size=(2,2),strides = (2, 2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPool2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256,(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides = (2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512,(3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides = (2, 2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512,(3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512,(3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512,(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides = (2, 2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))
#categorical
    
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### 查看model架构



In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     7

### 定义ImageDataGenerator


In [4]:
train_data_dir = r'D:/AI/11.6/dogs-vs-cats\train'
validation_data_dir = r'D:/AI/11.6/dogs-vs-cats\validation'
nb_train_samples = 10835
nb_validation_samples = 4000
epochs = 1
batch_size = 20


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



Found 10835 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


### 训练模型



In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

 93/541 [====>.........................] - ETA: 1:20:21 - loss: 7.1225 - accuracy: 0.5355

### 使用训练后模型预测图像





In [48]:
import cv2
img = cv2.resize(cv2.imread(r'D:/AI/11.6/dogs-vs-cats\test\7.jpg'), (img_width, img_height)).astype(np.float32)
# img[:,:,0] -= 103.939
# img[:,:,1] -= 116.779
# img[:,:,2] -= 123.68
#img = img.transpose((2,0,1))
x = img_to_array(img)

x = np.expand_dims(x, axis=0)

#x = preprocess_input(x)

score = model.predict(x)


print(score)

[[0.99895906]]
